> https://geckopy.readthedocs.io/en/latest/01-getting-started.html

In [ ]:
import cobra
from cobra.io.yaml import from_yaml
from cobra.io.sbml import write_sbml_model
import geckopy

## Import models

In [ ]:
yeast_ecgem_yaml = r"/home/daiki/Github/ENEOS/python/FBA/models/ecYeastGEM (SysBioChalmersGECKO).yml"
yeast_ecgem_xml = r"/home/daiki/Github/ENEOS/python/FBA/models/ecYeastGEM (SysBioChalmersGECKO).xml"

# YAMLを読み込み
yaml_model = from_yaml(open(yeast_ecgem).read())
# SBMLで保存
write_sbml_model(yaml_model, yeast_ecgem_xml)
# geckopyでSBMLを読み込み
ec_model = geckopy.io.read_sbml_ec_model(yeast_ecgem_xml)

## Python API: Protein isolation

In [ ]:
solution_rxn, solution_prot = ec_model.optimize() # cobrapyの solution オブジェクトを返す

# ec_model.slim_optimize() は目的関数の値（成長率など）を返すだけなので高速

In [ ]:
solution_rxn # 各反応のフラックス (.fluxes)

In [ ]:
solution_prot # タンパク質濃度とタンパク質貢献度 (.contribution)

## Kcats

- geckopy では、各酵素-反応ペアに対応する触媒定数 ($k_{cat}$) を `Protein`オブジェクト経由で **辞書型 (dict)** として扱う。
- $k_{cat}$ の入力単位はユーザー側が指定するのは $1/s$ $(秒^{-1})$ だが、内部では $1/h$ $(時間^{-1})$ に変換されている。
$$
k_{cat} [s^{-1}] \times 3600 \to k_{cat} [h^{-1}]
$$

In [ ]:
# kcat の参照と操作
print(ec_model.proteins.prot_P0A796.kcats) # Proteinオブジェクト(prot_P0A796)のkcatを確認
print(ec_model.proteins.prot_P0A796.kcats["PFKNo1"]) # PFK反応に対応する kcat

ec_model.proteins.prot_P0A796.kcats["PFKNo1"] = 60 # 新たに kcat を設定

In [ ]:
# 化学量論行列 S の量論係数 s を取得
ec_model.reactions.PFKNo1.metabolites[ec_model.proteins.prot_P0A796] # - 1/kcat (実際は - 1/(kcat*3600) の値が出力される) 

In [ ]:
ec_model.proteins.prot_P0A796.kcats["PFKNo1"] = 120 # 元の値が60に対して2倍の kcat を設定すれば、s は 1/2倍される
ec_model.reactions.PFKNo1.metabolites[ec_model.proteins.prot_P0A796]